In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

#### Valores Municipais

In [3]:
# Filtragem inicial para classe de mosaico de usos
df_munic_21 = (
    pd.read_parquet(f'{raiz}/prata_mapbiomas_uso_terra_municipio.parquet')
    .reset_index()
    .query('classe == 21 & data >= 2000')
)

lista_21 = (
    df_munic_21
    .geocodigo.drop_duplicates()
    .to_list()
)

In [4]:
df_munic = (
    pd.read_parquet(f'{raiz}/prata_mapbiomas_uso_terra_municipio.parquet')
    .reset_index()
    .query('classe in (15, 41, 48) & data >= 2000 & geocodigo in @lista_21')
    .assign(area = lambda n: pd.to_numeric(n.area, errors='coerce'))
    .convert_dtypes()
    .reset_index(drop=True)
)

In [5]:
df_munic_frac = (
    df_munic
    .assign(
        area_total_mun=lambda df: df.groupby(['geocodigo', 'data'])['area']
                                   .transform('sum'),
        frac_mun=lambda df: (df.area / df.area_total_mun).where(df.area_total_mun != 0, df.area)
    )
    .drop(columns=['area', 'area_total_mun'])
)

#### Valores Estaduais

In [6]:
# Cálculo das frações por UF direto dos dados municipais
df_uf_frac = (
    df_munic
    .assign(
        geocodigo_uf = lambda df: df.geocodigo // 100000,
        area_total_uf = lambda df: df.groupby(['geocodigo_uf', 'data'])['area']
                                  .transform('sum')
    )
    .assign(frac_uf = lambda df: df.area / df.area_total_uf)
    .drop(columns=['area', 'area_total_uf'])
    .filter(['geocodigo', 'geocodigo_uf', 'data', 'classe', 'frac_uf'])
)

#### Dados Restantes

In [7]:
# Identificação dos municípios restantes (que têm classe 21 mas não têm as outras classes)
municipios_restantes = list(set(lista_21) - set(df_munic_frac['geocodigo'].unique()))

# Estados dos municípios restantes (que têm classe 21 mas não têm as outras classes)
ufs_restantes = list(map(lambda x: int(str(x)[:2]), municipios_restantes))

In [8]:
# Processamento das frações por UF para municípios restantes
df_uf_rest_frac = (
    df_munic
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .query('classe in (15, 41, 48) & geocodigo_uf in @ufs_restantes')
    .assign(
        area_total_uf=lambda df: df.groupby(['geocodigo_uf', 'data'])['area']
                                  .transform('sum'),
        frac_uf=lambda df: df['area'] / df['area_total_uf']
    )
    .drop(columns=['area', 'area_total_uf'])
    .filter(['geocodigo_uf', 'data', 'classe', 'frac_uf'])
    .reset_index(drop=True)
)

In [9]:
# Aplicação das frações para municípios restantes (usando apenas dados estaduais)
df_resultado_restantes = (
    df_munic_21
    .query('geocodigo in @municipios_restantes')
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .merge(
        df_uf_rest_frac,
        on=['geocodigo_uf', 'data'],
        how='left',
        suffixes=['_x', '']
    )
    .assign(area_final = lambda df: df.area * df.frac_uf)
    .drop(columns=['classe_x', 'area', 'frac_uf'])
    .filter(['geocodigo', 'data', 'classe', 'area_final'])
)

#### Consolidação Principal

In [10]:
# Aplicação das frações para municípios com dados municipais
df_resultado_principais = (
    df_munic_21
    .query('geocodigo not in @municipios_restantes')
    .merge(
        df_munic_frac,
        on=['geocodigo', 'data'],
        how='left',
        suffixes=['_x', '']
    )
    .merge(
        df_uf_frac,
        on=['geocodigo', 'data', 'classe'],
        how='left'
    )
    .assign(
        frac_final=lambda df: df.frac_mun.fillna(df.frac_uf).where(lambda x: x != 0, df.frac_uf),
        area_final=lambda df: df.area * df.frac_final
    )
    .filter(['geocodigo', 'data', 'classe', 'area_final'])
)

In [11]:
# Combinação dos resultados
df_resultado = (
    pd.concat([
    df_resultado_principais,
    df_resultado_restantes
])
    .convert_dtypes()
    .sort_values(['geocodigo', 'data'])
    .reset_index(drop=True)
)

#### Nova tabela de uso da terra

In [19]:
df_uso_terra = (
    pd.read_parquet(f'{raiz}/prata_mapbiomas_uso_terra_municipio.parquet')
    .reset_index()
    .query('data >= 2000 & classe != 21')
    .merge(
        df_resultado,
        on=['geocodigo', 'data', 'classe'],
        how='left'
    )
    .assign(valor = lambda x: (x.area_final + x.area).where(x.area_final != np.nan, x.area))
    .dropna(subset='valor')
    .query('valor != 0')
    .convert_dtypes()
    .filter(['geocodigo', 'data', 'classe', 'valor'])
    .reset_index(drop=True)
)

df_uso_terra.to_parquet(f'{raiz}/ouro_is_agro_usodaterra.parquet', index=False)

In [20]:
df_br = (
    df_uso_terra
    .groupby(['data', 'classe'])
    .agg({'valor': 'sum'})
)

df_br.to_parquet(f'{raiz}/ouro_is_agro_usodaterra_br.parquet')